In [1]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger, LpStatus, value

In [2]:
# Sets
flights = ['F1', 'F2']
classes = ['Economy', 'Business', 'First']

# Parameters
# Demand
D = {
    ('F1', 'Economy'): 90, ('F1', 'Business'): 40, ('F1', 'First'): 10,
    ('F2', 'Economy'): 80, ('F2', 'Business'): 30, ('F2', 'First'): 20
}

# Capacity
K = {
    ('F1', 'Economy'): 100, ('F1', 'Business'): 50, ('F1', 'First'): 20,
    ('F2', 'Economy'): 90,  ('F2', 'Business'): 40, ('F2', 'First'): 30
}

# Distance
d_f = {'F1': 1000, 'F2': 1500}

# Base fare
c_seat = {
    ('F1', 'Economy'): 50, ('F1', 'Business'): 100, ('F1', 'First'): 150,
    ('F2', 'Economy'): 45, ('F2', 'Business'): 90,  ('F2', 'First'): 140
}


# c_luggage = {
#     ('F1', 'Economy'): 10, ('F1', 'Business'): 15, ('F1', 'First'): 20,
#     ('F2', 'Economy'): 10, ('F2', 'Business'): 15, ('F2', 'First'): 20
# }
# c_legroom = {
#     ('F1', 'Economy'): 5,  ('F1', 'Business'): 10, ('F1', 'First'): 15,
#     ('F2', 'Economy'): 5,  ('F2', 'Business'): 10, ('F2', 'First'): 15
# }

# Overbooking Cost
c_overbook = {
    ('F1', 'Economy'): 200, ('F1', 'Business'): 300, ('F1', 'First'): 500,
    ('F2', 'Economy'): 200, ('F2', 'Business'): 300, ('F2', 'First'): 500
}

# Unsold Cost
c_unsold = {
    ('F1', 'Economy'): 100, ('F1', 'Business'): 150, ('F1', 'First'): 200,
    ('F2', 'Economy'): 100, ('F2', 'Business'): 150, ('F2', 'First'): 200
}

# Overbooking Limit
alpha = {'Economy': 0.05, 'Business': 0.10, 'First': 0.05}

# Fuel Cost
c_fuel = 5

In [3]:
# Initialize model
model = LpProblem("Airline_Seat_Optimization", LpMinimize)

# Decision variables
X = LpVariable.dicts("X", [(f, c) for f in flights for c in classes], lowBound=0, cat=LpInteger)
O = LpVariable.dicts("O", [(f, c) for f in flights for c in classes], lowBound=0, cat=LpInteger)
U = LpVariable.dicts("U", [(f, c) for f in flights for c in classes], lowBound=0, cat=LpInteger)

# Objective function
model += lpSum(
    (c_seat[f, c]) * X[f, c] +
    c_overbook[f, c] * O[f, c] +
    c_unsold[f, c] * U[f, c]
    for f in flights for c in classes
) + lpSum(c_fuel * d_f[f] for f in flights)

# model += lpSum(
#     (c_seat[f, c] + c_luggage[f, c] + c_legroom[f, c]) * X[f, c] +
#     c_overbook[f, c] * O[f, c] +
#     c_unsold[f, c] * U[f, c]
#     for f in flights for c in classes
# ) + lpSum(c_fuel * d_f[f] for f in flights)

# Constraints
for f in flights:
    for c in classes:
        model += X[f, c] <= D[f, c] + O[f, c], f"Demand_Fulfillment_{f}_{c}"
        model += X[f, c] + U[f, c] == D[f, c] + O[f, c], f"Seat_Accounting_{f}_{c}"
        model += O[f, c] <= alpha[c] * D[f, c], f"Overbooking_Limit_{f}_{c}"
        model += X[f, c] <= K[f, c], f"Capacity_{f}_{c}"


In [4]:
# Solve
model.solve()

1

In [5]:
# Output
print("Status:", LpStatus[model.status])
print("Total Cost:", value(model.objective))
for v in model.variables():
    print(f"{v.name} = {v.varValue}")

Status: Optimal
Total Cost: 31600.0
O_('F1',_'Business') = 0.0
O_('F1',_'Economy') = 0.0
O_('F1',_'First') = 0.0
O_('F2',_'Business') = 0.0
O_('F2',_'Economy') = 0.0
O_('F2',_'First') = 0.0
U_('F1',_'Business') = 0.0
U_('F1',_'Economy') = 0.0
U_('F1',_'First') = 0.0
U_('F2',_'Business') = 0.0
U_('F2',_'Economy') = 0.0
U_('F2',_'First') = 0.0
X_('F1',_'Business') = 40.0
X_('F1',_'Economy') = 90.0
X_('F1',_'First') = 10.0
X_('F2',_'Business') = 30.0
X_('F2',_'Economy') = 80.0
X_('F2',_'First') = 20.0
